In [ ]:
"""
Draw a graph of the net architecture.
"""
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
from google.protobuf import text_format

import caffe
import caffe.draw
from caffe.proto import caffe_pb2
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image

def parse_args():
    """Parse input arguments
    """

    parser = ArgumentParser(description=__doc__,
                            formatter_class=ArgumentDefaultsHelpFormatter)

    #parser.add_argument('input_net_proto_file',
    #                    help='Input network prototxt file')
    #parser.add_argument('output_image_file',
    #                    help='Output image file')
    parser.add_argument('--rankdir',
                        help=('One of TB (top-bottom, i.e., vertical), '
                              'RL (right-left, i.e., horizontal), or another '
                              'valid dot option; see '
                              'http://www.graphviz.org/doc/info/'
                              'attrs.html#k:rankdir'),
                        default='LR')
    parser.add_argument('--phase',
                        help=('Which network phase to draw: can be TRAIN, '
                              'TEST, or ALL.  If ALL, then all layers are drawn '
                              'regardless of phase.'),
                        default="ALL")

    args = parser.parse_args()
    return args

class args_cls(object):
    def __init__(self):
        self.input_net_proto_file = '/home/ocean1101/Workspace/ssd/caffe/examples/AlexNet/cifar10/train_val.prototxt'
        self.phase = "TRAIN"
        '''
        rankdir : {'LR', 'TB', 'BT'}
        Direction of graph layout.
        '''
        self.rankdir = "TB"
        self.output_image_file = 'netimg.png'
        
def main():
    #args = parse_args()
    args = args_cls()
    net = caffe_pb2.NetParameter()
    text_format.Merge(open(args.input_net_proto_file).read(), net)
    #print('Drawing net to %s' % args.output_image_file)
    phase=None;
    if args.phase == "TRAIN":
        phase = caffe_pb2.TRAIN
    elif args.phase == "TEST":
        phase = caffe_pb2.TEST
    elif args.phase != "ALL":
        raise ValueError("Unknown phase: " + args.phase)
    caffe.draw.draw_net_to_file(net, args.output_image_file, args.rankdir,
                                phase)
    #print net, args.rankdir, phase
    #graph = caffe.draw.draw_net_to_file(net, args.rankdir, phase=phase)
    #img=Image.frombytes('RGB',sizeof(graph),graph)
    img = Image.open(args.output_image_file)
    
    print img.format,img.size, img.mode
    #import pdb;pdb.set_trace()
    
    img = np.array(img)
    plt.figure(figsize=(100,200))
    plt.imshow(img)#,aspect='auto')
    plt.show()

if __name__ == '__main__':
    main()


> /usr/local/lib/python2.7/dist-packages/pydot.py(1985)create()
-> cmdline = [self.progs[prog], '-T'+format, tmp_name] + args


In [49]:
import caffe
from __future__ import print_function
from caffe import layers as L, params as P, to_proto

def conv_bn_relu(bottom, kernel_size, num_output, stride=1, pad=0, group=1):
    conv = L.Convolution(bottom, kernel_size=kernel_size, stride=stride,
                                num_output=num_output, pad=pad, group=group,
                                weight_filler=dict(type='xavier'),
                                bias_filler=dict(type='constant', value=0))
    bn = L.BatchNorm(conv, use_global_stats=False)
    scale = L.Scale(bn, bias_term=True)
    relu = L.ReLU(scale, in_place=True)
    return conv, relu
#global average pooling
def fc_gav(bottom):
    #fc = L.InnerProduct(bottom, num_output=num_output)
    fc = L.Pooling(bottom, pool=P.Pooling.AVE, kernel_size=6, stride=1)
    bn = L.BatchNorm(fc, use_global_stats=False)
    scale = L.Scale(bn, bias_term=True)
    relu = L.ReLU(scale, in_place=True)
    drop = L.Dropout(relu, dropout_ratio=0.5)
    return drop

def max_pool(bottom, kernel_size, stride=2):
    return L.Pooling(bottom, pool=P.Pooling.MAX, kernel_size=kernel_size, stride=stride)

def mynet(lmdb, batch_size=256, include_acc=False):
    n = caffe.NetSpec()
    # data layer
    n.data, n.label = L.Data(source=lmdb, backend=P.Data.LMDB, batch_size=batch_size, ntop=2,
        transform_param=dict(crop_size=227, mean_value=[104, 117, 123], mirror=True))
    # the net itself
    #import pdb;pdb.set_trace()
    
    n.conv1, n.relu1 = conv_bn_relu(n.data, 3, 96, stride=1, pad=2)
    
    n.norm1 = L.LRN(n.relu1, local_size=5, alpha=1e-4, beta=0.75)
    n.pool1 = max_pool(n.norm1, 3, stride=2)
    n.conv2, n.relu2 = conv_bn_relu(n.pool1, 3, 128, stride=1, pad=2, group=2)
    n.norm2 = L.LRN(n.relu2, local_size=5, alpha=1e-4, beta=0.75)
    n.pool2 = max_pool(n.norm2, 3, stride=2)
    
    n.conv3, n.relu3 = conv_bn_relu(n.pool2, 3, 256, pad=2)
    
    n.conv4, n.relu4 = conv_bn_relu(n.relu3, 3, 384, pad=1, group=2)
    n.conv5, n.relu5 = conv_bn_relu(n.relu4, 3, 256, pad=1, group=2)
    
    n.pool5 = max_pool(n.relu5, 3, stride=2)
    
    n.fc7 = fc_gav(n.pool5)
    #fc8 = L.InnerProduct(fc7, num_output=10, kernel_size=1, stride=1,
    #    weight_filler=dict(type='xavier'), bias_filler=dict(type='constant',value=0))
    ''''''
    n.fc8 = L.Convolution(n.fc7, kernel_size=1, num_output=10,stride=1,
            weight_filler=dict(type='xavier'),
            bias_filler=dict(type='constant',value=0))
    
    n.acc = L.Accuracy(n.fc8, n.label)
    n.loss = L.SoftmaxWithLoss(n.fc8, n.label)
    return to_proto(n.loss, n.acc)

def make_net():
    #with open('train_val1.prototxt','w') as f:
        #print(mynet('123'), file=f)
    print(mynet('123'))

if __name__ == '__main__':
    make_net()


<type 'str'>
Data
test in layer_fn
name: Data  args: () kwargs: {'source': '123', 'ntop': 2, 'transform_param': {'mean_value': [104, 117, 123], 'mirror': True, 'crop_size': 227}, 'batch_size': 256, 'backend': 1}
<type 'str'>
Convolution
test in layer_fn
name: Convolution  args: (<caffe.net_spec.Top object at 0x7f50d1e03c90>,) kwargs: {'group': 1, 'num_output': 96, 'weight_filler': {'type': 'xavier'}, 'stride': 1, 'pad': 2, 'bias_filler': {'type': 'constant', 'value': 0}, 'kernel_size': 3}
<type 'str'>
BatchNorm
test in layer_fn
name: BatchNorm  args: (<caffe.net_spec.Top object at 0x7f50d1e03150>,) kwargs: {'use_global_stats': False}
<type 'str'>
Scale
test in layer_fn
name: Scale  args: (<caffe.net_spec.Top object at 0x7f50d1e03a50>,) kwargs: {'bias_term': True}
<type 'str'>
ReLU
test in layer_fn
name: ReLU  args: (<caffe.net_spec.Top object at 0x7f50d1e03290>,) kwargs: {'in_place': True}
<type 'str'>
LRN
test in layer_fn
name: LRN  args: (<caffe.net_spec.Top object at 0x7f50d1e030d0>